In [ ]:
#Team 27
#Κινους Βασιλειος Αλεξανδρος
#8834

In [ ]:
#Import the libraries that will be needed


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import colors
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler,RobustScaler,normalize
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.decomposition import PCA
import hyperopt
from hyperopt import hp
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import  accuracy_score, f1_score,precision_score,recall_score
import time
from umap import UMAP
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
from imblearn.combine import SMOTEENN
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

#Supress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv ('C:/Users/alexk/Downloads/datasetC.csv',header = None)

In [ ]:
#rename the target column

df = df.rename(columns={400: "Class"})

In [ ]:
#Target column counts (slight imbalance)

plt.figure(figsize=(8,8))
sns.set_style("darkgrid")
df["Class"].value_counts().plot.bar(rot=0)
plt.title("Value counts of Class column",fontsize = 20)
plt.xlabel("Class",fontsize = 12)
plt.ylabel("Count",fontsize = 12)
plt.savefig("C:/Users/alexk/Desktop/protipa/valuecountsexercise4.png", bbox_inches='tight')
plt.show()

In [ ]:
X = df.drop('Class', axis=1)

y = df['Class']

In [ ]:
y[y==1] = 0 
y[y==2] = 1 
y[y==3] = 2 
y[y==4] = 3 
y[y==5] = 4 

In [ ]:
#Create an explained variance ratio plot to check if pca is meaningful

pca_ratios = []
for i in range(300):
    #define PCA model to use
    pca = PCA(n_components=i,random_state = 77)

    #fit PCA model to data
    pca_fit = pca.fit_transform(X)
    
    pca_ratios.append(pca.explained_variance_ratio_.sum())    

In [ ]:
plt.figure(figsize =(8,8))    

closest_value = min(pca_ratios, key=lambda x: abs(x - 0.8))
index_of_closest_value = pca_ratios.index(closest_value)


g = sns.lineplot(data=pca_ratios)
g.axhline(0.8,ls='--',c="red")
g.axvline(index_of_closest_value,ls='--',c="red")


plt.scatter(index_of_closest_value, closest_value, color='red', zorder=5)

plt.xlabel("Components",fontsize=12)
plt.ylabel("Explained Variance",fontsize=12)
plt.title("Explained variance of different number of components",fontsize=18)
plt.savefig("C:/Users/alexk/Desktop/protipa/explainedvariancesum80.png", bbox_inches='tight')
plt.show()

In [ ]:
#Dataset Ctest has 1000 rows which is 20% of DatasetC so split 0.2% 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
#First lets try with a knn classifier with a tuning on its neighbors number.

knn_predictions = []
accuracy_matrix = []
knn_classifier = []
for neighbors in range(1,16):
    
    knn = KNeighborsClassifier(n_neighbors=neighbors)
    knn.fit(X_train, y_train)

    y_pred_knn = knn.predict(X_test)
    
    knn_classifier.append(knn)
    
    knn_predictions.append(y_pred_knn)
    
    accuracy_matrix.append(accuracy_score(y_test, y_pred_knn))
    
plt.figure(figsize = (8,8))
g = sns.lineplot(data=accuracy_matrix,marker="o",markersize = 12)
g.set_xticks(range(len(accuracy_matrix)))
g.set_xticklabels(np.arange(1,16),fontsize = 12)
plt.title("Accuracy of different neighbors",fontsize = 20)

plt.scatter(accuracy_matrix.index(max(accuracy_matrix)),max(accuracy_matrix), color='red', zorder=5)

#region of interest for the exercise because there are more points created to show the accuracy drop
#plt.axvspan(0, 9, color='blue', alpha=0.2)

for i in range(15) :

    g.text(i+0.12,accuracy_matrix[i]-0.0015,"N ={}".format(i+1))
    
plt.savefig("C:/Users/alexk/Desktop/protipa/knnaccuraciesexercise4", bbox_inches='tight')    
plt.show()

print("Best achieved accuracy with knn classifier is {}% with N-Neighbors = {}".format(100*(max(accuracy_matrix)),accuracy_matrix.index(max(accuracy_matrix))+1))

best_preds_knn = knn_predictions[accuracy_matrix.index(max(accuracy_matrix))]

print('Knn with N-Neighbors = {} model accuracy score: {}'.format(accuracy_matrix.index(max(accuracy_matrix))+1,accuracy_score(y_test, best_preds_knn)))
print('Knn with N-Neighbors = {} model precision score: {}'.format(accuracy_matrix.index(max(accuracy_matrix))+1,precision_score(y_test, best_preds_knn,average="weighted")))
print('Knn with N-Neighbors = {} model recall score: {}'.format(accuracy_matrix.index(max(accuracy_matrix))+1,recall_score(y_test, best_preds_knn,average="weighted")))
print('Knn with N-Neighbors = {} model f1 score: {}'.format(accuracy_matrix.index(max(accuracy_matrix))+1,f1_score(y_test, best_preds_knn,average="weighted")))

In [ ]:

#now try svm rbf kernel tuned

C_range_ = [1, 1e2,1e3]
gamma_range_ = [1e-2,1e-1, 1]
svm_classifiers = []
svm_predictions = []
accuracy_matrix_svm = []
for C in C_range_:
    for gamma in gamma_range_:
        svm_rbf = SVC(C=C, gamma=gamma)
        svm_rbf.fit(X_train, y_train)
        svm_classifiers.append(svm_rbf)
        
        y_pred_svm = svm_rbf.predict(X_test)
    
        
    
        svm_predictions.append(y_pred_svm)
    
        accuracy_matrix_svm.append(accuracy_score(y_test, y_pred_svm))
        
plt.figure(figsize = (8,8))
g = sns.lineplot(data=accuracy_matrix_svm,marker="o",markersize = 12)
g.set_xticks(range(len(accuracy_matrix_svm)))
g.set_xticklabels(np.arange(1,10),fontsize = 12)
plt.title("Accuracy of different C and gamma combinations",fontsize = 20)

plt.scatter(accuracy_matrix_svm.index(max(accuracy_matrix_svm)),max(accuracy_matrix_svm), color='red', zorder=5)

for i in range(9) :
    
    C_ = int(svm_classifiers[i].get_params()["C"])
    gamma_ = svm_classifiers[i].get_params()["gamma"]
    
    g.text(i+0.14,accuracy_matrix_svm[i]-0.007,"C ={}\nG ={}".format(C_,gamma_))

plt.savefig("C:/Users/alexk/Desktop/protipa/svmaccuraciesexercise4", bbox_inches='tight') 
    
plt.show()

print("Best achieved accuracy with tuned rbf kernel svm is {} with C ={} and gamma ={}".format(max(accuracy_matrix_svm),(svm_classifiers[accuracy_matrix_svm.index(max(accuracy_matrix_svm))].get_params()["C"]),(svm_classifiers[accuracy_matrix_svm.index(max(accuracy_matrix_svm))].get_params()["gamma"])))

best_preds_svm = svm_predictions[accuracy_matrix_svm.index(max(accuracy_matrix_svm))]

print('SVM best model accuracy score: {}'.format(accuracy_score(y_test, best_preds_svm)))
print('SVM best model precision score: {}'.format(precision_score(y_test, best_preds_svm,average="weighted")))
print('SVM best model recall score: {}'.format(recall_score(y_test, best_preds_svm,average="weighted")))
print('SVM best model f1 score: {}'.format(f1_score(y_test, best_preds_svm,average="weighted")))

In [ ]:
#stepwise Hyperopt tuning of xgboost with nested crossvalidation

#Keep n iterations constant and learning rate at 0.1 and experiment with the other hyperparameters first.

# begin with max depth and min_child_weight . Comment out the rest on space and keep them stock on xgbclassifier


#sample model to fire up the gpu so we can measure time elapsed more accurately
fire_up = XGBClassifier(n_estimators = 2000,tree_method = "hist",device="cuda")
fire_up.fit(X_train,y_train)


# Define the objective function for Hyperopt
def objective(space):
    # Create the XGBoost model with the specified hyperparameters running on gpu
    model = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,
                          
                          n_estimators =5000, 
                          max_depth= int(space["max_depth"])
                          ,min_child_weight= space["min_child_weight"]
                          
                          ,learning_rate = 0.1)
                          

    # Set up cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    

    # Train and evaluate the model for each fold
    scores = []
    for i , (train_index, val_index) in enumerate(cv.split(X_train, y_train)):
        # Split the data into training and evaluation sets
        X_train_check, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_check, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        evaluation = [( X_train_check, y_train_check), ( X_val, y_val)]

        # Train the model on the training data and use validation data for early stopping
        model.fit(X_train_check, y_train_check,eval_metric="mlogloss",eval_set=evaluation,early_stopping_rounds = 500)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model on the test data
        score = f1_score(y_test, y_pred, average="micro")
        scores.append(score)

    # Return the negative average of the scores
    return -np.mean(scores)

# Define the hyperparameter space for Hyperopt
space={  'max_depth': hp.quniform("max_depth", 2, 12, 1),
         #'gamma': hp.uniform ('gamma', 0,3),
         #'reg_alpha' : hp.quniform('reg_alpha', 5,50,1),
         #'reg_lambda' : hp.uniform('reg_lambda', 0,1),
         #'colsample_bytree' : hp.uniform('colsample_bytree', 0.3,1),
         'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
         'n_estimators' : 10000,
         #'learning_rate' : hp.uniform('learning_rate', 0.01,0.75),
         #'subsample' : hp.uniform('subsample', 0.2,1),
         
     }

# Run Hyperopt optimization

start = time.time()
trials = hyperopt.Trials()
best_params = hyperopt.fmin(
    objective,
    space,
    algo=hyperopt.rand.suggest,
    max_evals=30,
    trials=trials,
    verbose=2
)
end=time.time()

# Print the best hyperparameters
print(best_params)
print(end-start)

In [ ]:
best_loss_so_far = []
time_elapsed = []

best_loss_so_far.append(0.7942)
time_elapsed.append(8511.638578891754)

In [ ]:
#stepwise Hyperopt tuning of xgboost with nested crossvalidation

#Keep n iterations constant and learning rate at 0.1 and experiment with the other hyperparameters first.

# Now update the previous values and follow with subsample and colsample bytree.


#sample model to fire up the gpu so we can measure time elapsed more accurately
fire_up = XGBClassifier(n_estimators = 2000,tree_method = "hist",device="cuda")
fire_up.fit(X_train,y_train)


# Define the objective function for Hyperopt
def objective(space):
    # Create the XGBoost model with the specified hyperparameters running on gpu
    model = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,random_state = 45,
                          
                          n_estimators =5000, 
                          max_depth= 6
                          ,min_child_weight= 7.0
                          ,subsample =space["subsample"]
                          ,colsample_bytree = space["colsample_bytree"]
                          ,learning_rate = 0.1)
                          

    # Set up cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    

    # Train and evaluate the model for each fold
    scores = []
    for i , (train_index, val_index) in enumerate(cv.split(X_train, y_train)):
        # Split the data into training and evaluation sets
        X_train_check, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_check, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        evaluation = [( X_train_check, y_train_check), ( X_val, y_val)]

        # Train the model on the training data and use validation data for early stopping
        model.fit(X_train_check, y_train_check,eval_metric="mlogloss",eval_set=evaluation,early_stopping_rounds = 500)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model on the test data
        score = f1_score(y_test, y_pred, average="micro")
        scores.append(score)

    # Return the negative average of the scores
    return -np.mean(scores)

# Define the hyperparameter space for Hyperopt
space={  #'max_depth': hp.quniform("max_depth", 2, 12, 1),
         #'gamma': hp.uniform ('gamma', 0,3),
         #'reg_alpha' : hp.quniform('reg_alpha', 5,50,1),
         #'reg_lambda' : hp.uniform('reg_lambda', 0,1),
         'colsample_bytree' : hp.uniform('colsample_bytree', 0.3,1),
         #'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
         'n_estimators' : 10000,
         #'learning_rate' : hp.uniform('learning_rate', 0.01,0.75),
         'subsample' : hp.uniform('subsample', 0.2,1),
         
     }

# Run Hyperopt optimization

start = time.time()
trials = hyperopt.Trials()
best_params = hyperopt.fmin(
    objective,
    space,
    algo=hyperopt.rand.suggest,
    max_evals=30,
    trials=trials,
    verbose=2
)
end=time.time()

# Print the best hyperparameters
print(best_params)
time_elapsed.append(end-start)
print(end-start)

In [ ]:
best_loss_so_far.append(0.80560)

In [ ]:
#stepwise Hyperopt tuning of xgboost with nested crossvalidation

#Keep n iterations constant and learning rate at 0.1 and experiment with the other hyperparameters first.

# Now update the previous values and follow with learning rate.


#sample model to fire up the gpu so we can measure time elapsed more accurately
fire_up = XGBClassifier(n_estimators = 2000,tree_method = "hist",device="cuda")
fire_up.fit(X_train,y_train)


# Define the objective function for Hyperopt
def objective(space):
    # Create the XGBoost model with the specified hyperparameters running on gpu
    model = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,random_state = 45,
                          
                          n_estimators =5000, 
                          max_depth= 6
                          ,min_child_weight= 7.0
                          ,subsample =0.4942336627346027
                          ,colsample_bytree = 0.3878027876119541
                          ,learning_rate = space["learning_rate"])
                          

    # Set up cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    

    # Train and evaluate the model for each fold
    scores = []
    for i , (train_index, val_index) in enumerate(cv.split(X_train, y_train)):
        # Split the data into training and evaluation sets
        X_train_check, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_check, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        evaluation = [( X_train_check, y_train_check), ( X_val, y_val)]

        # Train the model on the training data and use validation data for early stopping
        model.fit(X_train_check, y_train_check,eval_metric="mlogloss",eval_set=evaluation,early_stopping_rounds = 500)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model on the test data
        score = f1_score(y_test, y_pred, average="micro")
        scores.append(score)

    # Return the negative average of the scores
    return -np.mean(scores)

# Define the hyperparameter space for Hyperopt
space={  #'max_depth': hp.quniform("max_depth", 2, 12, 1),
         #'gamma': hp.uniform ('gamma', 0,3),
         #'reg_alpha' : hp.quniform('reg_alpha', 5,50,1),
         #'reg_lambda' : hp.uniform('reg_lambda', 0,1),
         #'colsample_bytree' : hp.uniform('colsample_bytree', 0.3,1),
         #'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
         'n_estimators' : 10000,
         'learning_rate' : hp.uniform('learning_rate', 0.01,0.75),
         #'subsample' : hp.uniform('subsample', 0.2,1),
         
     }

# Run Hyperopt optimization

start = time.time()
trials = hyperopt.Trials()
best_params = hyperopt.fmin(
    objective,
    space,
    algo=hyperopt.rand.suggest,
    max_evals=30,
    trials=trials,
    verbose=2
)
end=time.time()

# Print the best hyperparameters
print(best_params)
time_elapsed.append(end-start)
print(end-start)

In [ ]:
best_loss_so_far.append(0.80979)

In [ ]:
#stepwise Hyperopt tuning of xgboost with nested crossvalidation

#Keep n iterations constant and learning rate at 0.1 and experiment with the other hyperparameters first.

# Now update the previous values and follow with learning rate.


#sample model to fire up the gpu so we can measure time elapsed more accurately
fire_up = XGBClassifier(n_estimators = 2000,tree_method = "hist",device="cuda")
fire_up.fit(X_train,y_train)


# Define the objective function for Hyperopt
def objective(space):
    # Create the XGBoost model with the specified hyperparameters running on gpu
    model = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,random_state = 45,
                          
                          n_estimators =1000, 
                          max_depth= 6
                          ,min_child_weight= 7.0
                          ,subsample =0.4942336627346027
                          ,colsample_bytree = 0.3878027876119541
                          ,learning_rate = 0.029058263960152986
                          ,gamma = space["gamma"])
                          

    # Set up cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    

    # Train and evaluate the model for each fold
    scores = []
    for i , (train_index, val_index) in enumerate(cv.split(X_train, y_train)):
        # Split the data into training and evaluation sets
        X_train_check, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_check, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        evaluation = [( X_train_check, y_train_check), ( X_val, y_val)]

        # Train the model on the training data and use validation data for early stopping
        model.fit(X_train_check, y_train_check,eval_metric="mlogloss",eval_set=evaluation,early_stopping_rounds = 50)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model on the test data
        score = f1_score(y_test, y_pred, average="micro")
        scores.append(score)

    # Return the negative average of the scores
    return -np.mean(scores)

# Define the hyperparameter space for Hyperopt
space={  #'max_depth': hp.quniform("max_depth", 2, 12, 1),
         'gamma': hp.uniform ('gamma', 0,3),
         #'reg_alpha' : hp.quniform('reg_alpha', 5,50,1),
         #'reg_lambda' : hp.uniform('reg_lambda', 0,1),
         #'colsample_bytree' : hp.uniform('colsample_bytree', 0.3,1),
         #'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
         'n_estimators' : 10000,
         #'learning_rate' : hp.uniform('learning_rate', 0.01,0.75),
         #'subsample' : hp.uniform('subsample', 0.2,1),
         
     }

# Run Hyperopt optimization

start = time.time()
trials = hyperopt.Trials()
best_params = hyperopt.fmin(
    objective,
    space,
    algo=hyperopt.rand.suggest,
    max_evals=30,
    trials=trials,
    verbose=2
)
end=time.time()

# Print the best hyperparameters
print(best_params)
time_elapsed.append(end-start)
print(end-start)

In [ ]:
best_loss_so_far.append(0.80840)

In [ ]:
#time passed doing training in hours

np.array(time_elapsed).sum()/3600

In [ ]:
plt.figure(figsize = (8,8))
g = sns.lineplot(data=best_loss_so_far,marker="o",markersize = 12)
g.set_xticks(range(len(best_loss_so_far)))
#g.set_xticklabels(np.arange(1,16),fontsize = 12)
plt.title("Accuracy gains or losses per step",fontsize = 20)

steps = ["Max depth\nMinChildWeight","Subsample\nColsampleBytree","LearningRate","Gamma"]
for i in range(4) :

    g.text(i+0.1,best_loss_so_far[i]-0.0002,steps[i])
    
plt.xlabel("Steps",fontsize=15)
plt.ylabel("Accuracy",fontsize=15)
    
plt.savefig("C:/Users/alexk/Desktop/protipa/highlowaccuracy4", bbox_inches='tight')    
plt.show()

In [ ]:
#retrain without gamma on whole model 

In [ ]:
xgb_without_gamma = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,random_state = 45,
                          
                          n_estimators =10000, 
                          max_depth= 6
                          ,min_child_weight= 7.0
                          ,subsample =0.4942336627346027
                          ,colsample_bytree = 0.3878027876119541
                          ,learning_rate = 0.029058263960152986
                          )

evaluation = [( X_train, y_train), ( X_test, y_test)]

xgb_without_gamma.fit(X_train, y_train,eval_metric="mlogloss",eval_set=evaluation,early_stopping_rounds = 1000)

In [ ]:
y_pred_nogamma = xgb_without_gamma.predict(X_test)

# compute and print accuracy score




print('XGBoost model accuracy score: {}'.format(accuracy_score(y_test, y_pred_nogamma)))
print('XGBoost model precision score: {}'.format(precision_score(y_test, y_pred_nogamma,average="weighted")))
print('XGBoost model recall score: {}'.format(recall_score(y_test, y_pred_nogamma,average="weighted")))
print('XGBoost model f1 score: {}'.format(f1_score(y_test, y_pred_nogamma,average="weighted")))

In [ ]:
#Lets try to oversample the imbalanced classes

smenn = SMOTETomek(random_state = 56)

X_train_resample,y_train_resample = smenn.fit_resample(X_train,y_train)

In [ ]:
xgb_resampled = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,random_state = 45,
                          
                          n_estimators =10000, 
                          max_depth= 6
                          ,min_child_weight= 7.0
                          ,subsample =0.4942336627346027
                          ,colsample_bytree = 0.3878027876119541
                          ,learning_rate = 0.029058263960152986
                          )

evaluation = [( X_train_resample, y_train_resample), ( X_test, y_test)]

xgb_resampled.fit(X_train_resample, y_train_resample,eval_metric="mlogloss",eval_set=evaluation,early_stopping_rounds = 1000)

In [ ]:
y_pred_res = xgb_resampled.predict(X_test)

# compute and print accuracy score
#Slight Accuracy increase when balancing out the classes (shoul)

print('XGBoost-upsampled model accuracy score: {}'.format(accuracy_score(y_test, y_pred_res)))
print('XGBoost-upsampled model precision score: {}'.format(precision_score(y_test, y_pred_res,average="weighted")))
print('XGBoost-upsampled model recall score: {}'.format(recall_score(y_test, y_pred_res,average="weighted")))
print('XGBoost-upsampled model f1 score: {}'.format(f1_score(y_test, y_pred_res,average="weighted")))

In [ ]:
#create a confusion matrix for the best model which is xgboost with upsampling

cm = confusion_matrix(y_test,y_pred_res)

cm_df = pd.DataFrame(cm,
                     index = ["0","1","2","3","4"], 
                     columns = ["0","1","2","3","4"])

#Plotting the confusion matrix
plt.figure(figsize=(8,8))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix',fontsize=20)
plt.ylabel('Actal Values',fontsize=12)
plt.xlabel('Predicted Values',fontsize=12)

plt.savefig("C:/Users/alexk/Desktop/protipa/confusionmatrix", bbox_inches='tight') 
plt.show()

In [ ]:
#The final model will be retrained on the whole dataset (X,y (no split)) and predict on the external DatasetC

In [ ]:
final_prediction_df = pd.read_csv ('C:/Users/alexk/Downloads/datasetCtest.csv',header = None)

In [ ]:
#we will use value around these (if they are close) for iterations of final model

print(xgb_without_gamma.best_iteration,xgb_resampled.best_iteration)


In [ ]:
#train final model on whole X y upsampled

smenn = SMOTETomek(random_state = 56)

X_train_resample_full,y_train_resample_full = smenn.fit_resample(X,y)


xgb_final = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,random_state = 45,
                          
                          n_estimators =1930, 
                          max_depth= 6
                          ,min_child_weight= 7.0
                          ,subsample =0.4942336627346027
                          ,colsample_bytree = 0.3878027876119541
                          ,learning_rate = 0.029058263960152986
                          )

xgb_final.fit(X_train_resample_full,y_train_resample_full)

In [ ]:
final_predictions = xgb_final.predict(final_prediction_df)

In [ ]:
# change from 0-1-2-3-4 to 1-2-3-4-5

final_predictions = np.array(final_predictions) +1

In [ ]:
np.save("C:/Users/alexk/Desktop/protipa/labels27",final_predictions)

In [ ]:
#double check if everything works

check_preds = np.load("C:/Users/alexk/Desktop/protipa/labels27.npy")

check_preds.shape

if check_preds.all() == final_predictions.all() :
    
    print("All good loaded array shape is {}".format(check_preds.shape))

In [ ]:


#Now we will try to do a 2d decision boundary plot like previous exercises. We will create a 2d embedding of the 400 features using UMAP

pca = PCA(n_components=2)

#reducer = UMAP(n_neighbors=100,n_components=2,random_state=55)

X_train_embedding = pca.fit_transform(X_train,y = y_train)

# Never fit the test data only transform to not cause data leakage !!!
X_test_embedding = pca.transform(X_test)

xgb_umap = XGBClassifier(objective="multi:huber",tree_method = "hist",device="cuda",n_classes = 5,random_state = 45,
                          
                          n_estimators =10000, 
                          max_depth= 6
                          ,min_child_weight= 7.0
                          ,subsample =0.4942336627346027
                          ,colsample_bytree = 0.3878027876119541
                          ,learning_rate = 0.029058263960152986
                          )

evaluation = [( X_train_embedding, y_train), ( X_test_embedding, y_test)]

xgb_umap.fit(X_train_embedding, y_train,eval_metric="mlogloss",eval_set=evaluation,early_stopping_rounds = 1000,verbose=False)

umap_predss = xgb_umap.predict(X_test_embedding)



print(accuracy_score(y_test, umap_predss))


fig = plt.figure(figsize=(12,12))

#sns.set(context="notebook", style="darkgrid",rc={"axes.axisbelow": False})

custom_cmap =  ListedColormap(['red','green','blue','yellow','black','purple'])

#sns.set_style="darkgrid"

#Use the already fitted model to create the decision surface by making predictions 
#for a grid of values across the input domain

#Find the minimum and maximum values for each feature and expand 
#the grid one step beyond that to ensure the whole feature space is covered.
x_min, x_max = X_test_embedding[:, 0].min() - 1, X_test_embedding[:, 0].max() + 1
y_min, y_max = X_test_embedding[:, 1].min() - 1, X_test_embedding[:, 1].max() + 1


#Create a uniform sample across each dimension using the linspace() function and make it really fine with 2000 points
#Use the meshgrid() function to create a grid from these two vectors.
xx, yy = np.meshgrid(np.linspace(x_min, x_max,2000),np.linspace(y_min, y_max,2000))

#Transform meshgrid to an expected form for the predict function. Flatten arrays with ravel function.
#Create a 2d array from the two 1d arrays using np.c_ function . Then we can use predict without errors.
Z = xgb_umap.predict(np.c_[xx.ravel(), yy.ravel()]) 

#Reshape to the starting meshgrid shape and plot the predicted contour.
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=custom_cmap,alpha = 0.5,zorder = 0)


# Seperate right wrong predictions to plot them with a different symbol

tp = (y_test == umap_predss)  # True Positive
tp0, tp1,tp2,tp3,tp4 = tp[y_test == 0], tp[y_test == 1] ,tp[y_test ==2], tp[y_test == 3] ,tp[y_test ==4]
X0, X1,X2,X3,X4 = X_test_embedding[y_test == 0], X_test_embedding[y_test == 1],X_test_embedding[y_test==2],X_test_embedding[y_test == 3],X_test_embedding[y_test==4]
X0_tp, X0_fp = X0[tp0], X0[~tp0]
X1_tp, X1_fp = X1[tp1], X1[~tp1]
X2_tp, X2_fp = X2[tp2], X2[~tp2]
X3_tp, X3_fp = X3[tp3], X3[~tp3]
X4_tp, X4_fp = X4[tp4], X4[~tp4]

#plt.scatter(X_test.iloc[:, 0], X_test.iloc[:, 1], c=y_test,cmap=custom_cmap,edgecolor="k",marker="o")


plt.scatter(X0_tp[:, 0], X0_tp[:, 1], c="red",edgecolor="k",marker="o",label = "0")
plt.scatter(X0_fp[:, 0], X0_fp[:, 1], c="red",marker="x",label = "0 missed")

plt.scatter(X1_tp[:, 0], X1_tp[:, 1], c="green",edgecolor="k",marker="o",label = "1")
plt.scatter(X1_fp[:, 0], X1_fp[:, 1], c="green",marker="x",label = "1 missed")

plt.scatter(X2_tp[:, 0], X2_tp[:, 1], c="blue",edgecolor="k",marker="o",label = "2")
plt.scatter(X2_fp[:, 0], X2_fp[:, 1], c="blue",marker="x",label = "2 missed")

plt.scatter(X3_tp[:, 0], X3_tp[:, 1], c="black",edgecolor="k",marker="o",label = "3")
plt.scatter(X3_fp[:, 0], X3_fp[:, 1], c="black",marker="x",label = "3 missed")

plt.scatter(X4_tp[:, 0], X4_tp[:, 1], c="purple",edgecolor="k",marker="o",label = "4")
plt.scatter(X4_fp[:, 0], X4_fp[:, 1], c="purple",marker="x",label = "4 missed")



#ax = fig.add_subplot(111, projection='3d')
#ax.plot_surface(xx, yy, Z, cmap='hsv', alpha=0.3)



plt.axis("off")

plt.title("Decision Boundary plot of XGB",fontsize = 20)


plt.legend()


plt.savefig("C:/Users/alexk/Desktop/protipa/decisionxgboost", bbox_inches='tight') 


plt.show()